In [1]:
import sys
import os
sys.path.append(os.path.abspath("./MLP"))
sys.path.append(os.path.abspath("./datasets"))
sys.path.append(os.path.abspath("./KAN"))
from MLP import MLP
from MLPWrapper import MLPWrapper
from convert_data_to_input import CreateInput
from KAN import KANWrapper
from KAN_model import KANModel
import torch
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np
import matplotlib.pyplot as plt
from sklearn import neural_network,pipeline,preprocessing,linear_model
import pandas as pd
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Prepare data

In [2]:
# get input data
deepmimo = False

function_folder = "uniform_sin(x)_241121"

data = CreateInput(function_folder, deepmimo)

tensors = data.get_tensors()

In [3]:
tensors['train'][1].shape

torch.Size([700])

# Evaluate KAN

In [ ]:
# find optimal params for KAN

kan_wrapper = KANWrapper(data=tensors,deepmimo=deepmimo)

if deepmimo:
    param_grid = {
        'kan__data': [tensors],
        'kan__width': [[1024]],
        'kan__grid': [3],
        'kan__k': [5],
        'kan__seed': [42],
        'kan__lr': [0.001],
        'kan__lamb': [0],
        'kan__deepmimo': [True]
    }

else:
    param_grid = {
        'kan__data': [tensors],
        'kan__width': [[1, 5, 1], [1, 3, 3, 1], [1, 7, 1], [1, 3, 3, 3, 1], [1, 3, 1], [1, 5, 5, 1], [1, 5, 5, 5, 1], [1, 7, 7, 1], [1, 7, 7, 7, 1]],
        'kan__grid': [3],
        'kan__k': [5],
        'kan__seed': [42],
        'kan__lr': [0.001, 0.0001],
        'kan__lamb': [0],
        'kan__deepmimo': [False],
        'kan__epochs': [20]
    }


# (Optional) Create a pipeline if preprocessing is needed
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('kan', kan_wrapper)
])

# Initialize RandomizedSearchCV
"""grid_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=5,                           # Number of parameter settings sampled
    cv=5,                                  # 5-Fold Cross-Validation
    scoring='neg_mean_squared_error',  # Appropriate for regression
    random_state=42,                    # For reproducibility
    n_jobs=-1,                          # Utilize all CPU cores
    verbose=2
)"""
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,                                  # 5-Fold Cross-Validation
    scoring='neg_mean_squared_error',  # Appropriate for regression
    n_jobs=-1,                          # Utilize all CPU cores
    verbose=0
)
# Fit RandomizedSearchCV
grid_search.fit(kan_wrapper.X_train, kan_wrapper.y_train)

# Retrieve the best parameters and best score
grid_search.best_params_['kan__data'] = []          # remove annoying print
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

opt_params_KAN = grid_search.best_params_


Fitting 5 folds for each of 18 candidates, totalling 90 fits


/Users/zeshenbao/KAN-Kolmogorov-Arnold-Networks/KAN/KAN.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _dataset['train_label'] = torch.tensor(y).float()
description:   0%|                                                           | 0/20 [00:00<?, ?it/s]/Users/zeshenbao/KAN-Kolmogorov-Arnold-Networks/KAN/KAN.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _dataset['train_label'] = torch.tensor(y).float()
/Users/zeshenbao/KAN-Kolmogorov-Arnold-Networks/KAN/KAN.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _dataset['train_l

[CV] END kan__data={'train': (tensor([[ 0.8308,  0.4702],
        [-1.1912, -0.3677],
        [-0.3504,  0.1642],
        ...,
        [ 7.2172,  0.9194],
        [-1.2913, -0.0462],
        [-7.9580, -0.9745]]), tensor([ 0.7385, -0.9288, -0.3432, -0.9998, -0.6500,  0.4130,  0.1709,  0.9992,
         0.6639,  0.9760,  0.9489,  0.5734,  0.9982, -0.8482, -0.8578, -0.8022,
         0.7498, -0.1875, -0.9905,  0.5033, -0.8449, -0.6845,  0.7259,  0.5388,
        -0.6651,  0.1298,  0.0116, -0.2862, -0.8878, -0.8695, -0.8031,  0.5909,
        -0.9669,  0.5218,  0.5440,  0.5073, -0.8374,  0.8140,  0.9224,  0.6057,
        -0.4354, -0.9424, -0.2700, -0.9371, -0.8570, -0.9998,  0.8679,  0.9750,
        -0.9413,  0.5859, -0.0931,  0.9805,  0.9980, -0.7216,  0.8586, -0.8886,
         0.7529, -0.9982,  0.3258, -0.7406, -0.9992,  0.9992, -0.8490, -0.8366,
        -0.4201,  0.0147, -0.7227,  0.1465, -0.7891,  0.6989, -0.3273, -0.8263,
        -0.7920, -0.2506,  0.4396, -0.9796, -0.4173, -0.7747,  0.93

| train_loss: 4.16e-01 | test_loss: 4.31e-01 | reg: 0.00e+00 | :  50%|▌| 10/20 [00:02<00:02,  4.56it/Users/zeshenbao/KAN-Kolmogorov-Arnold-Networks/KAN/KAN.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _dataset['train_label'] = torch.tensor(y).float()
| train_loss: 4.25e-01 | test_loss: 4.32e-01 | reg: 1.19e+01 | : 100%|█| 20/20 [00:03<00:00,  6.06it
| train_loss: 4.25e-01 | test_loss: 4.33e-01 | reg: 0.00e+00 | :  95%|▉| 19/20 [00:03<00:00,  7.48it

[CV] END kan__data={'train': (tensor([[ 0.8308,  0.4702],
        [-1.1912, -0.3677],
        [-0.3504,  0.1642],
        ...,
        [ 7.2172,  0.9194],
        [-1.2913, -0.0462],
        [-7.9580, -0.9745]]), tensor([ 0.7385, -0.9288, -0.3432, -0.9998, -0.6500,  0.4130,  0.1709,  0.9992,
         0.6639,  0.9760,  0.9489,  0.5734,  0.9982, -0.8482, -0.8578, -0.8022,
         0.7498, -0.1875, -0.9905,  0.5033, -0.8449, -0.6845,  0.7259,  0.5388,
        -0.6651,  0.1298,  0.0116, -0.2862, -0.8878, -0.8695, -0.8031,  0.5909,
        -0.9669,  0.5218,  0.5440,  0.5073, -0.8374,  0.8140,  0.9224,  0.6057,
        -0.4354, -0.9424, -0.2700, -0.9371, -0.8570, -0.9998,  0.8679,  0.9750,
        -0.9413,  0.5859, -0.0931,  0.9805,  0.9980, -0.7216,  0.8586, -0.8886,
         0.7529, -0.9982,  0.3258, -0.7406, -0.9992,  0.9992, -0.8490, -0.8366,
        -0.4201,  0.0147, -0.7227,  0.1465, -0.7891,  0.6989, -0.3273, -0.8263,
        -0.7920, -0.2506,  0.4396, -0.9796, -0.4173, -0.7747,  0.93

...

| train_loss: 4.25e-01 | test_loss: 4.32e-01 | reg: 9.52e+00 | : 100%|█| 20/20 [00:02<00:00,  7.14it
| train_loss: 4.25e-01 | test_loss: 4.32e-01 | reg: 0.00e+00 | :  95%|▉| 19/20 [00:02<00:00,  8.99it/Users/zeshenbao/KAN-Kolmogorov-Arnold-Networks/KAN/KAN.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _dataset['train_label'] = torch.tensor(y).float()
| train_loss: 4.25e-01 | test_loss: 4.32e-01 | reg: 9.57e+00 | : 100%|█| 20/20 [00:02<00:00,  7.30it
| train_loss: 4.23e-01 | test_loss: 4.38e-01 | reg: 0.00e+00 | :  65%|▋| 13/20 [00:04<00:02,  2.83it

[CV] END kan__data={'train': (tensor([[ 0.8308,  0.4702],
        [-1.1912, -0.3677],
        [-0.3504,  0.1642],
        ...,
        [ 7.2172,  0.9194],
        [-1.2913, -0.0462],
        [-7.9580, -0.9745]]), tensor([ 0.7385, -0.9288, -0.3432, -0.9998, -0.6500,  0.4130,  0.1709,  0.9992,
         0.6639,  0.9760,  0.9489,  0.5734,  0.9982, -0.8482, -0.8578, -0.8022,
         0.7498, -0.1875, -0.9905,  0.5033, -0.8449, -0.6845,  0.7259,  0.5388,
        -0.6651,  0.1298,  0.0116, -0.2862, -0.8878, -0.8695, -0.8031,  0.5909,
        -0.9669,  0.5218,  0.5440,  0.5073, -0.8374,  0.8140,  0.9224,  0.6057,
        -0.4354, -0.9424, -0.2700, -0.9371, -0.8570, -0.9998,  0.8679,  0.9750,
        -0.9413,  0.5859, -0.0931,  0.9805,  0.9980, -0.7216,  0.8586, -0.8886,
         0.7529, -0.9982,  0.3258, -0.7406, -0.9992,  0.9992, -0.8490, -0.8366,
        -0.4201,  0.0147, -0.7227,  0.1465, -0.7891,  0.6989, -0.3273, -0.8263,
        -0.7920, -0.2506,  0.4396, -0.9796, -0.4173, -0.7747,  0.93

/Users/zeshenbao/KAN-Kolmogorov-Arnold-Networks/KAN/KAN.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _dataset['train_label'] = torch.tensor(y).float()
| train_loss: 4.21e-01 | test_loss: 4.33e-01 | reg: 9.31e+00 | : 100%|█| 20/20 [00:02<00:00,  7.32it
| train_loss: 4.26e-01 | test_loss: 4.33e-01 | reg: 0.00e+00 | :  60%|▌| 12/20 [00:04<00:03,  2.66it/Users/zeshenbao/KAN-Kolmogorov-Arnold-Networks/KAN/KAN.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _dataset['train_label'] = torch.tensor(y).float()
description:   0%|                                                           | 0/20 [00:00<?, ?it/s]

[CV] END kan__data={'train': (tensor([[ 0.8308,  0.4702],
        [-1.1912, -0.3677],
        [-0.3504,  0.1642],
        ...,
        [ 7.2172,  0.9194],
        [-1.2913, -0.0462],
        [-7.9580, -0.9745]]), tensor([ 0.7385, -0.9288, -0.3432, -0.9998, -0.6500,  0.4130,  0.1709,  0.9992,
         0.6639,  0.9760,  0.9489,  0.5734,  0.9982, -0.8482, -0.8578, -0.8022,
         0.7498, -0.1875, -0.9905,  0.5033, -0.8449, -0.6845,  0.7259,  0.5388,
        -0.6651,  0.1298,  0.0116, -0.2862, -0.8878, -0.8695, -0.8031,  0.5909,
        -0.9669,  0.5218,  0.5440,  0.5073, -0.8374,  0.8140,  0.9224,  0.6057,
        -0.4354, -0.9424, -0.2700, -0.9371, -0.8570, -0.9998,  0.8679,  0.9750,
        -0.9413,  0.5859, -0.0931,  0.9805,  0.9980, -0.7216,  0.8586, -0.8886,
         0.7529, -0.9982,  0.3258, -0.7406, -0.9992,  0.9992, -0.8490, -0.8366,
        -0.4201,  0.0147, -0.7227,  0.1465, -0.7891,  0.6989, -0.3273, -0.8263,
        -0.7920, -0.2506,  0.4396, -0.9796, -0.4173, -0.7747,  0.93

| train_loss: 4.25e-01 | test_loss: 4.38e-01 | reg: 9.35e+00 | : 100%|█| 20/20 [00:02<00:00,  7.07it
| train_loss: 4.21e-01 | test_loss: 4.50e-01 | reg: 0.00e+00 | :  60%|▌| 12/20 [00:04<00:03,  2.63it/Users/zeshenbao/KAN-Kolmogorov-Arnold-Networks/KAN/KAN.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _dataset['train_label'] = torch.tensor(y).float()
| train_loss: 4.21e-01 | test_loss: 4.50e-01 | reg: 0.00e+00 | :  65%|▋| 13/20 [00:04<00:02,  2.70it

In [ ]:
# evaluate KAN with optimal params
kan_model = KANModel(width=opt_params_KAN['kan__width'], grid=opt_params_KAN['kan__grid'], k=opt_params_KAN['kan__k'], seed=opt_params_KAN['kan__seed'], lr=opt_params_KAN['kan__lr'], lamb=opt_params_KAN['kan__lamb'], deepmimo=deepmimo, epochs=100)
kan_model.load_data(tensors)
loss_data, time_elapsed = kan_model.fit()
y_preds = kan_model.predict()

In [ ]:
if deepmimo:
    kan_model = KANModel(width=[1024, 10,10, 1024], grid=20, k=3, seed=42, lr=0.1, lamb=0,deepmimo=deepmimo,steps=10)
    kan_model.load_data(tensors)
    loss_data, time_elapsed = kan_model.fit()
    y_preds = kan_model.predict()

In [ ]:
# plot pred and true
if deepmimo:
    kan_model.plot_deepmimo(data=tensors, y_preds=y_preds['preds'], type_='test', save=False)
else:
    kan_model.plot_prediction(data=tensors, y_preds=y_preds['preds'], type_='test', save=False)

In [ ]:
# plot noise and true
if deepmimo:
    kan_model.plot_deepmimo(data=tensors, y_preds=tensors['test'][0][:2,:], type_='test', save=False)

In [ ]:
# diffrence in noise and true
#torch.sum(np.abs(tensors['test'][0][:1,:] - tensors['test'][1][:1,:]))

In [ ]:
# diffrence pred and true
#torch.sum(np.abs(y_preds['preds'][0] - tensors['test'][1][:1,:]))

In [ ]:
# plot loss
kan_model.plot_loss(loss_data=loss_data, save=False)

# Evaluate MLP

In [ ]:
# find optimal params for mlp
mlp_wrapper = MLPWrapper(data=tensors,deepmimo=deepmimo)

if deepmimo:
    param_grid = {
        'mlp__data': [tensors],
        'mlp__input_size': [1024],
        'mlp__hidden_size': [[64,64],[500,500],[2000],[1000,1000],[300,300,300],[5000],[3000,3000],[32,32],[128,128],[256,64]],
        'mlp__output_size': [1024],
        'mlp__epochs': [100],
        'mlp__deepmimo': [True]
    }

else:
    param_grid = {
        'mlp__data': [tensors],
        'mlp__input_size': [1],
        'mlp__hidden_size': [[3], [3, 3], [3, 3, 3], [5], [5, 5], [5, 5, 5], [7], [7, 7], [7, 7, 7]],
        'mlp__output_size': [1],
        'mlp__epochs': [20, 21],
        'mlp__deepmimo': [False],
        'mlp__lr': [0.001]
    }


# (Optional) Create a pipeline if preprocessing is needed
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Example preprocessor
    ('mlp', mlp_wrapper)
])

# Initialize RandomizedSearchCV
grid_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=5,  # Number of parameter settings sampled
    cv=5,        # 5-Fold Cross-Validation
    scoring='neg_mean_squared_error',  # Appropriate for regression
    random_state=42,                    # For reproducibility
    n_jobs=-1,                          # Utilize all CPU cores
    verbose=0
)

# Fit RandomizedSearchCV
grid_search.fit(mlp_wrapper.X_train, mlp_wrapper.y_train)

# Retrieve the best parameters and best score
grid_search.best_params_['mlp__data'] = []          # remove annoying print
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

opt_params_MLP = grid_search.best_params_

In [ ]:
# evaluate KAN with optimal params
mlp_model = MLP(input_size=opt_params_MLP['mlp__input_size'], hidden_sizes=opt_params_MLP['mlp__hidden_size'], output_size=opt_params_MLP['mlp__output_size'])
mlp_model.load_data(tensors, deepmimo=deepmimo)
loss_data_MLP, time_elapsed_MLP = mlp_model.fit(mlp_model.X_train, mlp_model.y_train, n_epochs=100, cross_validation=False, deepmimo=deepmimo)
y_preds_MLP = mlp_model.predict(mlp_model.X_test,eval=True)

In [ ]:
# plot predictions
if deepmimo:
    mlp_model.plot_deepmimo(data=tensors, y_preds=y_preds_MLP['preds'], type_='test', save=False)
else:
    mlp_model.plot_prediction(data=tensors, y_preds=y_preds_MLP['preds'], type_='test', save=False)

In [ ]:
# plot loss

mlp_model.plot_loss(loss_data=loss_data_MLP, save=False, deepmimo=deepmimo)